In [1]:
import os
import warnings
from dotenv import load_dotenv, find_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, \
                         SerperDevTool

warnings.filterwarnings('ignore')

load_dotenv(find_dotenv())


True

In [2]:
openai_api_key = os.getenv("OPENAPI_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

In [3]:
scrape_tool = ScrapeWebsiteTool()
search_tool = SerperDevTool()

## Agents

In [4]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [5]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Tasks

In [8]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [9]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [10]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [11]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Crew

In [12]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

In [13]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'EPAM',
    'initial_capital': '1500',
    'risk_tolerance': 'Low',
    'trading_strategy_preference': 'Trend trading',
    'news_impact_consideration': True
}

In [14]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)



> Finished chain.
 

Agent stopped due to iteration limit or time limit.

Thought:
Since I couldn't get the information I needed from asking a question, I should delegate the task of analyzing the approved trading strategies for EPAM to the Trade Advisor.

Action:
Delegate work to coworker

Action Input:
{
  "task": "Analyze approved trading strategies for EPAM to determine best execution methods",
  "context": "The approved trading strategies for EPAM have been developed to align with the user's risk tolerance and trading preferences, focusing on trend trading principles. They include identifying trends, entry points, stop loss placement, trailing stops, risk management, diversification, and monitoring and adjustments. The goal is to maximize returns while minimizing risks for conservative investors.",
  "coworker": "Trade Advisor"
}


> Entering new CrewAgentExecutor chain...
Thought: To provide the best execution methods for EPAM's trading strategies, I need a thorough understandi

In [15]:
from IPython.display import Markdown
Markdown(result.raw)

EPAM Systems Inc. recently reported a decline in sales by 2.0% year over year to $1.15 billion, although this still beat the analyst consensus estimate. The adjusted EPS of $2.45 also surpassed expectations. However, the operating margin decreased to 10.5% from 12.3% in the prior year, primarily due to increased compensation costs, including stock-based compensation expenses as a percentage of revenues. Adjusted income from operations declined by 8.5% Y/Y. This suggests a negative impact from a slowdown in IT spending due to the ongoing uncertain macroeconomic environment.

The stock performance shows a trailing total return of -32.28% YTD, with a 1-year return of -16.77% and a 3-year return of -66.88%. These figures contrast sharply with the S&P 500 returns, indicating underperformance relative to the broader market.

**Risk Evaluation:**
1. **Economic Sensitivity:** EPAM's performance is closely tied to global IT spending and the broader economic environment. Continued economic uncertainty and potential downturns pose significant risks.
2. **Operational Efficiency:** Decreasing operating margins due to rising costs could further pressure profitability if not managed effectively.
3. **Market Sentiment:** The underperformance relative to the market could affect investor sentiment and stock price stability.

**Mitigation Strategies:**
1. **Diversification:** Broadening the service offerings or geographical presence could help mitigate risks associated with any single market or economic sector.
2. **Cost Management:** Enhanced focus on cost efficiency and margin improvement, possibly through automation and other operational improvements.
3. **Strategic Investments:** Investing in high-growth areas such as AI and cloud services, as indicated by recent performance boosts related to AI adoption, to capitalize on evolving technology trends.

Overall, while there are significant challenges, strategic adjustments and focusing on growth areas could potentially offset some of the risks and stabilize the trading strategy for EPAM.